In [34]:
from ib_insync import *

In [35]:
import nest_asyncio
nest_asyncio.apply()

from ib_insync import *

# Connect to IB Gateway or TWS
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)  # use 4002 for paper, 7496 for live

print("Connected:", ib.isConnected())

Connected: True


Error 200, reqId 7: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20251114', strike=262.5, right='C', multiplier='USD', exchange='SMART')
Error 200, reqId 8: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20251114', strike=262.5, right='C', multiplier='USD', exchange='SMART')
Error 200, reqId 11: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20251219', strike=262.5, right='C', multiplier='USD', exchange='SMART')
Error 200, reqId 12: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20251219', strike=262.5, right='C', multiplier='USD', exchange='SMART')
Error 10276, reqId 22: News feed is not allowed., contract: Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSy

In [67]:
from ib_insync import *
from datetime import datetime
from math import log, sqrt, exp
from scipy.stats import norm
import numpy as np

In [70]:
def black_scholes_call_price(S, K, T, r, sigma):
    if T <= 0:
        return max(S-K, 0)
    d1 = (log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    return S*norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)

def implied_vol_call(S, K, T, r, market_price, tol=1e-5, max_iter=100):
    sigma = 0.2  # initial guess
    for i in range(max_iter):
        price = black_scholes_call_price(S, K, T, r, sigma)
        vega = S * norm.pdf((log(S/K)+(r+0.5*sigma**2)*T)/(sigma*sqrt(T))) * sqrt(T)
        if vega == 0:
            return None
        diff = price - market_price
        if abs(diff) < tol:
            return sigma
        sigma -= diff/vega
        if sigma <= 0:
            sigma = tol
    return sigma

def get_chain(symbol):
    stock = Stock(symbol, 'SMART', 'USD')
    ib.qualifyContracts(stock)
    chains = ib.reqSecDefOptParams(stock.symbol, '', stock.secType, stock.conId)
    if not chains:
        return None
    chain = chains[0]
    strikes = sorted(chain.strikes)
    expirations = sorted([datetime.strptime(d, '%Y%m%d').date() for d in chain.expirations])
    return {'strikes': strikes, 'expirations': expirations}

def get_atm_iv(symbol, expiry):
    chain = get_chain(symbol)
    if not chain:
        return None
    stock = Stock(symbol, 'SMART', 'USD')
    ib.qualifyContracts(stock)
    ticker_stock = ib.reqMktData(stock)
    ib.sleep(1)
    S = ticker_stock.last or ticker_stock.close
    if not S:
        return None

    # pick closest strike that exists
    strikes = chain['strikes']
    strikes = [s for s in strikes if s > 0]
    if not strikes:
        return None
    atm_strike = min(strikes, key=lambda s: abs(s-S))

    # try known exchanges for OPRA options
    for exch in ['CBOE', 'SMART']:
        option = Option(symbol, expiry.strftime('%Y%m%d'), atm_strike, 'C', exch)
        ib.qualifyContracts(option)
        ticker_opt = ib.reqMktData(option)
        ib.sleep(1)
        bid = ticker_opt.bid
        ask = ticker_opt.ask
        if bid is not None and ask is not None:
            market_price = (bid+ask)/2
            T = (expiry - datetime.now().date()).days / 365
            r = 0.05
            return implied_vol_call(S, atm_strike, T, r, market_price)

    # if no valid quote found
    return None

def forward_vol(iv1, iv2, t1, t2):
    if iv1 is None or iv2 is None:
        return None
    return np.sqrt((iv2**2*t2 - iv1**2*t1)/(t2-t1))

def calc_forward_vol(symbol):
    chain = get_chain(symbol)
    if not chain:
        print(f"No option chain for {symbol}")
        return None
    today = datetime.now().date()
    exp30 = min(chain['expirations'], key=lambda d: abs((d-today).days - 30))
    exp60 = min(chain['expirations'], key=lambda d: abs((d-today).days - 60))
    iv30 = get_atm_iv(symbol, exp30)
    iv60 = get_atm_iv(symbol, exp60)
    fwd = forward_vol(iv30, iv60, 30/365, 60/365)
    print(f"{symbol} results:")
    print(f"  30D IV ({exp30}): {iv30 if iv30 else 'unavailable'}")
    print(f"  60D IV ({exp60}): {iv60 if iv60 else 'unavailable'}")
    print(f"  Forward Vol: {fwd if fwd else 'unavailable'}")
    return {'symbol': symbol, 'iv30': iv30, 'iv60': iv60, 'forward_vol': fwd}


In [71]:
tickers = ['AAPL','MSFT','GOOG']
results = []
for t in tickers:
    res = calc_forward_vol(t)
    if res:
        results.append(res)

Unknown contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20251219', strike=262.5, right='C', exchange='CBOE')


AAPL results:
  30D IV (2025-11-21): 0.28024726288324975
  60D IV (2025-12-19): nan
  Forward Vol: nan


Unknown contract: Option(symbol='MSFT', lastTradeDateOrContractMonth='20251219', strike=517.5, right='C', exchange='CBOE')


MSFT results:
  30D IV (2025-11-21): 0.2748034349259272
  60D IV (2025-12-19): nan
  Forward Vol: nan


Unknown contract: Option(symbol='GOOG', lastTradeDateOrContractMonth='20251219', strike=257.5, right='C', exchange='CBOE')


GOOG results:
  30D IV (2025-11-21): 0.4129982892386392
  60D IV (2025-12-19): nan
  Forward Vol: nan
